In [1]:
import pandas as pd
import numpy as np

# Đọc dữ liệu từ file Excel, với tên cột ở hàng đầu tiên
df = pd.read_excel("NFLX.xlsx", sheet_name="NFLX")

# In ra các tên cột để kiểm tra
print("Tên các cột trong dữ liệu:")
print(df.columns)

# X là ma trận dữ liệu, Y là vector kết quả
# X1 = Open, X2 = High, X3 = Low, X4 = Volume
# Y = Adj Close
X = df[["X1= Open(Giá mở cửa)", "X2= High(Giá cao nhất)", "X3= Low(Giá thấp nhất)", "X4= Volume(Khối lượng giao dịch)"]].values  # Chọn các cột Open, High, Low, Volume
X = np.hstack((np.ones((X.shape[0], 1)), X))  # Thêm cột 1 vào X để tính hệ số chệch

Y = df["Y= Adj Close( Giá đóng phiên điều chỉnh)"].values  # Cột Adj Close là Y

# Tính Beta mũ theo công thức: Beta_hat = (X^T X)^-1 X^T Y
XT = X.T
Beta_hat = np.linalg.inv(XT @ X) @ XT @ Y

# In ra kết quả
print("Beta mũ (ước lượng):")
print(Beta_hat)

# Hiển thị mô hình hồi quy dưới dạng công thức
feature_names = ["1"] + ["X1", "X2", "X3", "X4"]  # Tên các biến trong X (thêm hệ số chặn)
equation_terms = [f"{Beta_hat[i]:.11f} * {feature_names[i]}" for i in range(len(Beta_hat))]
model_equation = "y = " + " + ".join(equation_terms)

# In công thức hồi quy
print("Mô hình hồi quy (y_hat):")
print(model_equation)



Tên các cột trong dữ liệu:
Index(['X1= Open(Giá mở cửa)', 'X2= High(Giá cao nhất)',
       'X3= Low(Giá thấp nhất)', 'X4= Volume(Khối lượng giao dịch)',
       'Y= Adj Close( Giá đóng phiên điều chỉnh)'],
      dtype='object')
Beta mũ (ước lượng):
[-4.55323324e-01 -5.97693885e-01  7.72316330e-01  8.26377970e-01
  8.80383531e-08]
Mô hình hồi quy (y_hat):
y = -0.45532332372 * 1 + -0.59769388502 * X1 + 0.77231632991 * X2 + 0.82637796974 * X3 + 0.00000008804 * X4


In [2]:
from scipy.stats import f
# Số lượng mẫu (n) và số biến độc lập (k)
n = X.shape[0]
k = X.shape[1] - 1  # Trừ đi cột hệ số chặn

# Tính tổng bình phương sai số hồi quy (SS_R)
SS_R = Beta_hat.T @ XT @ Y - (np.sum(Y) ** 2) / n

# Tính tổng bình phương sai số đo lường sự thay đổi ngoài mô hình (SS_E)
SS_E = Y.T @ Y - Beta_hat.T @ XT @ Y

SS_T = SS_R + SS_E

# Tính MS_R và MS_E
MS_R = SS_R / k
MS_E = SS_E / (n - (k + 1))

# Tính giá trị quan sát f_0
f_0 = MS_R / MS_E

# Tính R^2
R_squared = SS_R / SS_T


# Tìm giá trị tới hạn f_{alpha}(4, 1004)
alpha = 0.05
df1 = k  # Bậc tự do của MS_R
df2 = n - (k + 1)  # Bậc tự do của MS_E
f_critical = f.ppf(1 - alpha, df1, df2)

# Kiểm tra f_0 có thuộc W_alpha hay không
if f_0 > f_critical:
    result = "f_0 thuộc W_alpha"
else:
    result = "f_0 không thuộc W_alpha"


# In kết quả
print("\nKết quả tính toán:")
print(f"Tổng bình phương sai số hồi quy (SS_R): {SS_R}")
print(f"Tổng bình phương sai số đo lường sự thay đổi ngoài mô hình (SS_E): {SS_E}")
print(f"Tổng bình phương sai số tổng thể mô hình(SS_T): {SS_T}")
print(f"MS_R (trung bình bình phương hồi quy): {MS_R}")
print(f"MS_E (trung bình bình phương sai số): {MS_E}")
print(f"Giá trị quan sát f_0: {f_0}")
print(f"R^2 (hệ số xác định): {R_squared}")

print("\nKết quả kiểm tra Fisher:")
print(f"f_0: {f_0}")
print(f"Giá trị tới hạn f_{alpha}(4, 1004): {f_critical}")
print(f"Kết luận: {result}")

# Tạo bảng ANOVA
anova_table = pd.DataFrame({
    "Nguồn": ["Hồi quy", "Sai số", "Tổng"],
    "Tổng bình phương": [SS_R, SS_E, SS_T],
    "Bậc tự do": [df1, df2, df1 + df2],
    "Binh phương TB": [MS_R, MS_E, ""],
    "f_0": [f_0, "", ""]
})

# In bảng ANOVA
print("\nBảng ANOVA:")
print(anova_table.to_string(index=False))

print(f"R^2 (hệ số xác định): {R_squared}")


Kết quả tính toán:
Tổng bình phương sai số hồi quy (SS_R): 11805402.195792347
Tổng bình phương sai số đo lường sự thay đổi ngoài mô hình (SS_E): 15135.428559541702
Tổng bình phương sai số tổng thể mô hình(SS_T): 11820537.624351889
MS_R (trung bình bình phương hồi quy): 2951350.548948087
MS_E (trung bình bình phương sai số): 15.075128047352294
Giá trị quan sát f_0: 195776.15126568987
R^2 (hệ số xác định): 0.9987195651297315

Kết quả kiểm tra Fisher:
f_0: 195776.15126568987
Giá trị tới hạn f_0.05(4, 1004): 2.380795469026456
Kết luận: f_0 thuộc W_alpha

Bảng ANOVA:
  Nguồn  Tổng bình phương  Bậc tự do  Binh phương TB            f_0
Hồi quy      1.180540e+07          4  2951350.548948  195776.151266
 Sai số      1.513543e+04       1004       15.075128               
   Tổng      1.182054e+07       1008                               
R^2 (hệ số xác định): 0.9987195651297315


In [3]:
XTX_inv = np.linalg.inv(XT @ X)

# In ma trận (X^T X)^-1
print("\nMa trận (X^T X)^-1:")
print(XTX_inv)


Ma trận (X^T X)^-1:
[[ 2.78442541e-02 -1.93873247e-06  1.84045528e-04 -2.37976652e-04
  -8.16796393e-10]
 [-1.93873247e-06  4.24395586e-05 -2.25319342e-05 -1.98977083e-05
   7.69778101e-13]
 [ 1.84045528e-04 -2.25319342e-05  4.60591821e-05 -2.45753319e-05
  -2.60894836e-11]
 [-2.37976652e-04 -1.98977083e-05 -2.45753319e-05  4.56485512e-05
   2.70019900e-11]
 [-8.16796393e-10  7.69778101e-13 -2.60894836e-11  2.70019900e-11
   6.01915319e-17]]


In [4]:
# Tính các giá trị T_j
C_diag = np.diag(np.linalg.inv(XT @ X))  # Lấy đường chéo chính của ma trận (X^T X)^-1
T_values = Beta_hat / np.sqrt(MS_E * C_diag)  # Tính T_j theo công thức

# In kết quả
print("Giá trị T_j:")
for j, T_j in enumerate(T_values):
    print(f"T_{j} = {T_j}")


Giá trị T_j:
T_0 = -0.7027835682503761
T_1 = -23.629954138053307
T_2 = 29.309378013030187
T_3 = 31.50175334473496
T_4 = 2.922629117635863


In [5]:
from scipy.stats import t

# Định nghĩa mức ý nghĩa alpha và bậc tự do
alpha = 0.05
df = n - (k + 1)  # Bậc tự do

# Tìm giá trị tới hạn t_{alpha/2; df} cho miền hai phía
t_critical = t.ppf(1 - alpha / 2, df)  # Vì phân phối đối xứng, chỉ cần t_critical bên phải

# Kiểm tra từng t_j có thuộc miền bác bỏ hay không
W_alpha_results = []
for j, T_j in enumerate(T_values):
    if T_j > t_critical or T_j < -t_critical:
        W_alpha_results.append(f"T_{j} thuộc W_alpha (|T_{j}| > {t_critical:.4f})")
    else:
        W_alpha_results.append(f"T_{j} không thuộc W_alpha (|T_{j}| ≤ {t_critical:.4f})")

# In kết quả
print("\nKiểm tra T_j thuộc miền bác bỏ W_alpha:")
print("\n".join(W_alpha_results))



Kiểm tra T_j thuộc miền bác bỏ W_alpha:
T_0 không thuộc W_alpha (|T_0| ≤ 1.9623)
T_1 thuộc W_alpha (|T_1| > 1.9623)
T_2 thuộc W_alpha (|T_2| > 1.9623)
T_3 thuộc W_alpha (|T_3| > 1.9623)
T_4 thuộc W_alpha (|T_4| > 1.9623)
